In [31]:
import pandas as pd
import numpy as np
import os
import csv
import ast
from collections import Counter
import geopandas as gpd
import h5py
from datetime import datetime, timedelta
#### personal packages####
import sys

In [32]:
cluster_name = {0: 'Glocery&Pharmacies', 1: 'Retails', 2: 'Arts&Entertainment', 3: 'Restaurants&Bars',
                4: 'Educations', 5: 'Healthcares', 6: 'others'}

In [33]:
df_flow = pd.read_csv('../results/outflow_7c_wk_raw.csv')
df_flow['MODZCTA'] = df_flow['MODZCTA'].astype(str)

In [34]:
df_flow.head()

,MODZCTA,mod_idx,Glocery&Pharmacies_0,Glocery&Pharmacies_1,Glocery&Pharmacies_2,Glocery&Pharmacies_3,Glocery&Pharmacies_4,Glocery&Pharmacies_5,Glocery&Pharmacies_6,Glocery&Pharmacies_7,...,others_46,others_47,others_48,others_49,others_50,others_51,others_52,others_53,others_54,others_55
0,10463,0,2487.655245,2275.680558,1843.889403,2651.257490,2188.490532,2013.783596,1658.860760,1073.727648,...,1559.042424,1311.936342,1324.153258,1384.974465,1257.324068,1375.229453,1002.906180,1480.891291,1407.041800,1306.053993
1,10471,1,623.722759,405.454625,612.218598,546.460340,530.761999,617.323880,337.775556,249.837528,...,458.574342,475.775682,359.265769,435.449029,335.950641,544.031862,382.070734,427.738877,433.704457,369.952058
2,10466,2,3886.352388,3285.403224,2836.591637,3907.839585,3194.343397,3506.359917,2207.903384,1547.531695,...,2953.330333,2907.669631,2867.336372,2638.141946,2457.859791,2678.608428,2151.994640,2447.586891,2250.649334,2198.777401
3,10469,3,3721.460268,3479.092721,3003.788466,3641.022828,3314.116577,3213.887501,2443.129012,1745.621272,...,2929.376979,2543.060451,2791.909151,2939.298600,2848.747022,2720.786523,2105.575169,2711.692493,2831.577121,2591.741624
4,10475,4,1976.166466,1986.236375,1825.521536,1755.846313,1774.496597,1930.771287,1389.960498,776.002776,...,2271.982737,2169.631134,2173.624303,2122.259775,2033.363970,2055.229514,1787.006269,1792.355506,1911.192523,2017.125010


In [35]:
cluster_cols = [cluster_name[0]+'_{}'.format(i) for i in range(56)] 
# 56 weeks from 2020-02-04 to 2021-03-02
# only need 53 weeks from 2020-02-24 to 2021-02-02
df_vist_cluster_i = df_flow[['MODZCTA','mod_idx']+cluster_cols].copy()

final_df = pd.melt(df_vist_cluster_i, id_vars=['MODZCTA','mod_idx'], value_vars = cluster_cols[3:3+51], var_name='week', value_name=cluster_name[0]+'_visits_weekly')
#### 
# Correctly adjust the code to perform the intended replacement
final_df['week'] = final_df['week'].apply(lambda x: (int(x.split('_')[1]) - 3))

for c_idx in range(1,7):
    print(cluster_name[c_idx])
    cluster_cols = [cluster_name[c_idx]+'_{}'.format(i) for i in range(56)]
    df_vist_cluster_i = df_flow[['MODZCTA', 'mod_idx']+cluster_cols].copy()
    temp_df = pd.melt(df_vist_cluster_i, id_vars=['MODZCTA', 'mod_idx'], value_vars = cluster_cols[3:3+51], 
                    var_name='week', value_name=cluster_name[c_idx]+'_visits_weekly')
    temp_df['week'] = temp_df['week'].apply(lambda x: (int(x.split('_')[1]) - 3))
    final_df = pd.merge(final_df, temp_df, on=['MODZCTA','mod_idx','week'])

Retails
Arts&Entertainment
Restaurants&Bars
Educations
Healthcares
others


In [36]:
### 1 covid_data_population file 
df_mod_borough = pd.read_csv('../Data/data-by-modzcta_11102020.csv')
# the len of the df_mod_borough is 177, the number of MODZCTA is 177
df_mod_borough['MODIFIED_ZCTA'] = df_mod_borough['MODIFIED_ZCTA'].astype(str)
replace_dic = dict(zip(['Manhattan', 'Staten Island', 'Bronx', 'Queens', 'Brooklyn'],['MN','SI','BX','QN','BK']))
df_mod_borough['borough'] = df_mod_borough['BOROUGH_GROUP'].replace(replace_dic)

for col in ['COVID_CASE_COUNT','COVID_DEATH_COUNT']:
    df_mod_borough[col] = df_mod_borough[col].astype(float)
    df_mod_borough.loc[df_mod_borough[col]==0,col] = 0.1
    df_mod_borough[col+'_log'] = np.log(df_mod_borough[col])

df_merged = final_df.merge(df_mod_borough, right_on = 'MODIFIED_ZCTA', left_on='MODZCTA')
columns_to_delete = ['MODIFIED_ZCTA','BOROUGH_GROUP']
df_merged = df_merged.drop(columns=columns_to_delete)
df_merged['MODZCTA'] = df_merged['MODZCTA'].astype(int)

In [37]:
len(df_merged)

9027

Merge all the other data at the zipcode level

In [38]:
### 2 read scores data at zipcode level#########
df_mod_score = pd.read_csv('../results/modzcta_scores.csv')
df_2 = df_merged.merge(df_mod_score, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
df_2.dropna(subset=['score_median'],inplace=True)
print(len(df_2))

6477


In [39]:
##### 3 social economic data ######
# df_sed_nyc = pd.read_csv('../Data/sed_nyc_modzcta.csv')
df_sed_nyc = pd.read_csv('../Data/sed_old.csv')
df_3 = df_2.merge(df_sed_nyc, right_on = 'MODZCTA', left_on='MODZCTA',how='left')
print(len(df_3))

6477


In [40]:
### 4 merge with borough cases ###
covid_cases = pd.read_csv('../data/cases_borough_weekly.csv')
for col in ['CASE_COUNT','DEATH_COUNT', 'borough_case_count', 'borough_death_count']:
    covid_cases[col] = covid_cases[col].astype(float)
    covid_cases.loc[covid_cases[col]==0,col] = 0.1
    covid_cases[col+'_log'] = np.log(covid_cases[col])

df_4 = df_3.merge(covid_cases, on=['borough', 'week'], how='left')
print(len(df_4))

6477


In [41]:
### 5 merge with policies ####
df_policies = pd.read_csv('../data/stringency_index_ny_wk.csv')
visits_scores_wk = df_4.merge(df_policies, on='week', how='left')
print(len(visits_scores_wk))

6477


In [43]:
visits_scores_wk.to_csv('../data/unpivot_merged_data_raw_old.csv',index=False)

In [30]:
visits_scores_wk.columns

Index(['MODZCTA', 'mod_idx', 'week', 'Glocery&Pharmacies_visits_weekly',
       'Retails_visits_weekly', 'Arts&Entertainment_visits_weekly',
       'Restaurants&Bars_visits_weekly', 'Educations_visits_weekly',
       'Healthcares_visits_weekly', 'others_visits_weekly',
       'NEIGHBORHOOD_NAME', 'COVID_CASE_COUNT', 'COVID_CASE_RATE',
       'POP_DENOMINATOR', 'COVID_DEATH_COUNT', 'COVID_DEATH_RATE',
       'PERCENT_POSITIVE', 'TOTAL_COVID_TESTS', 'borough',
       'COVID_CASE_COUNT_log', 'COVID_DEATH_COUNT_log', 'score_mean',
       'score_median', 'score_std', 'gain_bias_mean', 'gain_bias_median',
       'gain_bias_std', 'loss_bias_mean', 'loss_bias_median', 'loss_bias_std',
       'loss_aversion_scores_mean', 'loss_aversion_scores_median',
       'loss_aversion_scores_std', 'rescale_avail_mean',
       'rescale_avail_median', 'rescale_avail_std', 'rescale_realistic_mean',
       'rescale_realistic_median', 'rescale_realistic_std', 'agency_mean',
       'agency_median', 'agency_std',